<a href="https://colab.research.google.com/github/JuliethLopez/chatbot_simpsons/blob/master/Pre_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# librerias
import pickle
import re, string
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer

# 1.Clasificador
### Cargar modelo

In [19]:
!unzip '/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/clasificador.zip' #julieth

Archive:  /content/drive/My Drive/Simpsons Chat bot/Modelos guardos/clasificador.zip
   creating: clasificador/
   creating: clasificador/assets/
   creating: clasificador/variables/
  inflating: clasificador/variables/variables.data-00000-of-00002  
  inflating: clasificador/variables/variables.index  
  inflating: clasificador/variables/variables.data-00001-of-00002  
  inflating: clasificador/saved_model.pb  


In [20]:
modelo_clasificador = tf.keras.models.load_model('clasificador')
#Check its architecture
#modelo_clasificador.summary()

### Cargar para preprocesamiento
Cargamos el label_tokenizer, el encodeer, los trigramas y bigramas.

In [21]:
#label tokenizer
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/label_tokenizer.pickle', 'rb') as f:
  label_tokenizer = pickle.load(f)

In [22]:
#encoder
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/encoder.pickle', 'rb') as f:
  encoder = pickle.load(f)

In [23]:
#trigramas
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/freqnew.pickle', 'rb') as f:
  freqnew = pickle.load(f)

In [24]:
#bigramas
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/freqbnew.pickle', 'rb') as f:
  freqbnew = pickle.load(f)

### Función clasificadora

In [25]:
# funcion de frase tokenizada a palabras
reverse_word_map = dict(map(reversed, label_tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    # Busca palabras en el diccionario
    label = [reverse_word_map.get(i) for i in list_of_indices]
    return(label)

In [33]:
def clasifica (sentence):
  sentence = re.sub(' +', ' ',sentence.lower()) #sentence en minusculas
  sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence) #quita signos de puntuación
  # preprocesamiento trigramas
  vectorizer3 = CountVectorizer(vocabulary=freqnew.keys(), ngram_range=(3,3))
  X3 = vectorizer3.fit_transform([sentence])
  # preprocesamiento bigramas
  vectorizer2 = CountVectorizer(vocabulary=freqbnew.keys(), ngram_range=(2,2))
  X2 = vectorizer2.fit_transform([sentence])
  # concatenamos
  sequences_bt = np.concatenate((X3.toarray(), X2.toarray()), axis=1)
  # predicción
  pred = np.argmax(modelo_clasificador.predict(sequences_bt), axis=-1) #prediccion del modelo
  pred = encoder.inverse_transform(pred) #de label encoder a tokenizer
  pred = sequence_to_text(pred) #de tokenizer a label
  
  return (pred)

In [35]:
#ejemplo
sentence = "Would this be a good time to be honest?"
clasifica(sentence)

['homer_simpson']

# 2.Generador
### Cargar modelo

In [27]:
!unzip '/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/generador.zip' #julieth

Archive:  /content/drive/My Drive/Simpsons Chat bot/Modelos guardos/generador.zip
   creating: generador/
  inflating: generador/saved_model.pb  
   creating: generador/variables/
  inflating: generador/variables/variables.data-00000-of-00002  
  inflating: generador/variables/variables.index  
  inflating: generador/variables/variables.data-00001-of-00002  
   creating: generador/assets/


In [28]:
modelo_generador = tf.keras.models.load_model('generador')
#Check its architecture
#modelo_clasificador.summary()

### Cargar para preprocesamiento

In [29]:
# char a id
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/charxid.pickle', 'rb') as f:
  charxid = pickle.load(f)

In [30]:
#id a char
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/idxchar.pickle', 'rb') as f:
  idxchar = pickle.load(f)

### Función generadora

In [31]:
#funcion para generar texto usando el modelo pre-entrenado
def generate_text(start_string, num_generate):

  #convertimos el texto inicial en numeros (vectorizacion)
  input_eval = [charxid[s] for s in start_string] #vector columna
  input_eval = tf.expand_dims(input_eval, 0) #vector fila, agrega una diension

  #guardamos el texto predicho
  text_generated = []

  temperature = 1.0

  #tamaño del bache== 1
  modelo_generador.reset_states() #reinia o borra el estado recurrente de la red. Dejando valores aleatorios o ceros.
  for i in range(num_generate):
      predictions = modelo_generador(input_eval)
      #removemos la dimensión del batch, quita una dimensión
      predictions = tf.squeeze(predictions, 0)

      #usamos la distribución categorica para predecir la palabra que retorna el modelo
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      #utilizamos la palabra predicha y el estado oculto anterior como entrada
      input_eval = tf.expand_dims([predicted_id], 0) #devuelve un tensor con una dimensión adicional en el eje de índice

      text_generated.append(idxchar[predicted_id])

  return (start_string + ''.join(text_generated))

In [36]:
#ejemplo
sentence = "Would this be a good time to be honest?"
generate_text(sentence, 100)

"Would this be a good time to be honest?\r\nBart Simpson: Santgha vias. Well, I gave you to hold myself a can't get so lip comf the rubboard.\r"

#Función para unir redes

# Función para limpiar salida

# Pre chatbot

In [138]:
# frase del usuario
sentence = "Would this be a good time to be honest?"

['homer_simpson']

In [ ]:
def predictor(sentence, num_generate):
  clase = clasifica(sentence)
  start_string = union(clase, sentence)
  line = generate_text(start_string, num_generate)
  output_ultimate(line)